In [1]:
from openai import OpenAI
client = OpenAI(api_key="sk-proj-ZFMwsidUEKkbHLGwLaouT3BlbkFJfzqmBaNeeAtGZyAvYrgh")
client.embeddings.create(input = ["text"], model="text-embedding-3-small").data[0].embedding

[-0.005959634203463793,
 0.029575351625680923,
 -0.03841280937194824,
 -0.039994511753320694,
 0.03022811748087406,
 -0.031106842681765556,
 0.037760041654109955,
 -0.015578527934849262,
 0.0051185693591833115,
 -0.019520234316587448,
 -0.0002263499773107469,
 0.015879804268479347,
 -0.006502560339868069,
 0.0030629821121692657,
 0.022922152653336525,
 0.03216131031513214,
 0.022445131093263626,
 -0.028420455753803253,
 0.022420024499297142,
 0.03650471940636635,
 0.043634939938783646,
 0.0020681405439972878,
 -0.005231548100709915,
 -0.033943865448236465,
 -0.010262245312333107,
 0.025244494900107384,
 -0.041827280074357986,
 0.011184905655682087,
 0.06482475250959396,
 -0.03439578413963318,
 0.0052189952693879604,
 -0.036253657191991806,
 0.06321793794631958,
 -0.06979582458734512,
 0.04840515926480293,
 0.010732990689575672,
 0.031583864241838455,
 -0.01095894817262888,
 0.046597499400377274,
 -0.039216212928295135,
 -0.01728576421737671,
 -0.017298316583037376,
 0.00854245573282241